In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
#tqdm.pandas(desc='pandas bar')
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from gensim.models import Word2Vec
import gc
import time

#general purpose packages
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

#data processing
import re, string
import nltk

from sklearn import preprocessing
from sklearn.model_selection import train_test_split


#Naive Bayes
from sklearn.naive_bayes import MultinomialNB

#lightgbm
import lightgbm as lgb

#transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

#keras
import tensorflow as tf
from tensorflow import keras

#metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

#tqdm.pandas(desc='pandas bar')
import warnings
warnings.filterwarnings('ignore')

#set style for plots
sns.set_style("whitegrid")
sns.despine()
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlepad=10)

In [4]:
import os
os.getcwd()

In [5]:
train1 = pd.read_csv('../input/kuc-hackathon-winter-2018/drugsComTrain_raw.csv')
test1 = pd.read_csv('../input/kuc-hackathon-winter-2018/drugsComTest_raw.csv')
df = pd.concat([train1,test1])
df

In [7]:
df["rating"] = df["rating"].astype("int32")
df["usefulCount"] = df["usefulCount"].astype("int32")

In [8]:
# condition column only 0.55% percent missing values, we can drop it
df = df.dropna()

In [9]:
df.drop_duplicates(subset=["review"],inplace=True)

In [10]:
df

In [11]:
# sort the dataframe by uniqueID
df = df.sort_values("uniqueID").reset_index(drop=True)

In [12]:
# these 1171 rows have wrong condition info, but the reviews are still useful.
df[df["condition"].str.contains("</span>")]["condition"] = "Not Listed / Othe"

In [13]:
df.loc[:,"sentiment"] = ""
df.loc[df["rating"].isin([7,8,9,10]),"sentiment"] = 2
df.loc[df["rating"].isin([1,2,3]),"sentiment"] = 0
df.loc[df["rating"].isin([4,5,6]),"sentiment"] = 1

In [14]:

from nltk.tokenize import sent_tokenize # tokenizes sentences
from nltk.stem import PorterStemmer     # parsing/stemmer
from nltk.tag import pos_tag            # parts-of-speech tagging
from nltk.corpus import wordnet         # sentiment scores
from nltk.stem import WordNetLemmatizer # stem and context
from nltk.corpus import stopwords       # stopwords
from nltk.util import ngrams            # ngram iterator
from nltk.tokenize import word_tokenize


# import word2vec
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec

# import sklearn
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, FunctionTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bs4 import BeautifulSoup
import nltk
import re

In [15]:
ps = PorterStemmer()
wnl = WordNetLemmatizer()
eng_stopwords = set(stopwords.words("english"))


def review_cleaner(review, lemmatize=True, stem=False):
    '''
        Clean and preprocess a review.
            1. Remove HTML tags
            2. Extract emoticons
            3. Use regex to remove all special characters (only keep letters)
            4. Make strings to lower case and tokenize / word split reviews
            5. Remove English stopwords
            6. Lemmatize
            7. Rejoin to one string
        
        @review (type:str) is an unprocessed review string
        @return (type:str) is a 6-step preprocessed review string
    '''
    if lemmatize == True and stem == True:
        raise RuntimeError("May not pass both lemmatize and stem flags")

    #1. Remove HTML tags
    review = BeautifulSoup(review, "lxml").text

    #2. Use regex to find emoticons
    pattern2 ='(?::|;|=)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(pattern2,review)

    #3. Remove punctuation
    pattern3 = r"[^\w+ \d+]"
    review = re.sub(pattern3, "",review)

    #4. Tokenize into words (all lower case)
    review = review.lower()

    #5/6. Remove stopwords, Lemmatize, Stem
        
    word_tokens = word_tokenize(review)
    filtered_sentence = [w for w in word_tokens if not w in eng_stopwords]
    filtered_sentence = []
    for w in word_tokens:
         if w not in eng_stopwords:               
                w = wnl.lemmatize(w)              
                filtered_sentence.append(w)
                
    
    #7. Join the review to one sentence
    filtered_sentence = " ".join(filtered_sentence+emoticons)        
    
    return filtered_sentence


In [16]:
%time df['review_clean'] = df['review'].apply(review_cleaner)

In [17]:
df = df[['uniqueID','review_clean','sentiment']]

In [18]:
result = []
for text in df.review_clean:
        temp = len(text.split())
        result.append(temp)
        
df["text_len"] = result
df["text_len"]

In [19]:
df = df[df["text_len"]<=150]

In [20]:
test = df[df.uniqueID.isin(test1["uniqueID"].values)].reset_index(drop=True)
train = df[df.uniqueID.isin(train1["uniqueID"].values)].reset_index(drop=True)

In [21]:
df['sentiment'] = df['sentiment'].astype("int32")
train["sentiment"] = train["sentiment"].astype("int32")
test["sentiment"] = test["sentiment"].astype("int32")

In [22]:
X = train['review_clean'].values
y = train['sentiment'].values

In [23]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

In [24]:
train.sentiment.value_counts()

In [25]:
X_test = test['review_clean'].values
y_test = test['sentiment'].values

In [40]:

y_train_le = y_train.copy()
y_valid_le = y_valid.copy()
y_test_le = y_test.copy()

In [41]:
ohe = preprocessing.OneHotEncoder()
y_train = ohe.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_valid = ohe.fit_transform(np.array(y_valid).reshape(-1, 1)).toarray()
y_test = ohe.fit_transform(np.array(y_test).reshape(-1, 1)).toarray()

In [42]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [43]:
token_lens = []

for txt in train['review_clean'].values:
    tokens = tokenizer.encode(txt, max_length=256, truncation=True)
    token_lens.append(len(tokens))
    
max_len=np.max(token_lens)

In [44]:
print(f"MAX TOKENIZED SENTENCE LENGTH: {max_len}")

In [45]:
MAX_LEN=256

def tokenize(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [46]:
train_input_ids, train_attention_masks = tokenize(X_train, MAX_LEN)
val_input_ids, val_attention_masks = tokenize(X_valid, MAX_LEN)
test_input_ids, test_attention_masks = tokenize(X_test, MAX_LEN)

In [47]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

In [48]:
def create_model(bert_model, max_len=MAX_LEN):
    
    ##params###
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
    loss = tf.keras.losses.CategoricalCrossentropy()
    accuracy = tf.keras.metrics.CategoricalAccuracy()


    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    embeddings = bert_model([input_ids,attention_masks])[1]
    
    output = tf.keras.layers.Dense(3, activation="softmax")(embeddings)
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)
    
    model.compile(opt, loss=loss, metrics=accuracy)
    
    
    return model

In [49]:
model = create_model(bert_model, MAX_LEN)
model.summary()

In [50]:
np.mean([len(train_input_ids[i]) for i in range(len(train_input_ids))])

In [51]:
history_bert = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=4, batch_size=32)

In [52]:
result_bert = model.predict([test_input_ids,test_attention_masks])

In [53]:
y_pred_bert =  np.zeros_like(result_bert)
y_pred_bert[np.arange(len(y_pred_bert)), result_bert.argmax(1)] = 1

In [55]:
y_pred_bert

In [68]:
print('\tClassification Report for BERT:\n\n',classification_report(y_test,y_pred_bert, target_names=['Negative', 'Neutral', 'Positive']))

In [62]:
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
# I run it in Kaggle with GPU. but can't download ipynb file with output..